In [ ]:
# January-kenkyuu GPU/CPU フルパイプライン実行ノート

# このノートの目的:
# - Google Drive をマウント
# - リポジトリを clone して January-kenkyuu ブランチをチェックアウト
# - 依存パッケージをインストール
# - CLI ランナー (scripts/run_full_pipeline_cli.py) を CPU / GPU で実行

from pathlib import Path

print("Notebook for January-kenkyuu full pipeline (CPU/GPU) setup")

In [ ]:
!rm -rf /content/drive

In [ ]:
# 1. Google Drive をマウント

from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# 2. リポジトリを clone し、January-kenkyuu ブランチをチェックアウト

%cd /content

# TODO: あなたの GitHub リポジトリURL に書き換えてください
REPO_URL = "https://github.com/your-account/kenkyuu.git"

!git clone "$REPO_URL" kenkyuu
%cd /content/kenkyuu

!git fetch
!git checkout January-kenkyuu


In [ ]:
# 3. 依存パッケージのインストール

%pip install -q -r requirements.txt

# GPU 用ライブラリ（PyTorch）をインストール
# PyTorch は CUDA ランタイムを内包しているため、Colab 環境で動作しやすい
# 既に requirements.txt に含まれていますが、明示的にインストール
%pip install -q torch torchvision torchaudio

In [ ]:
# 4. GPU が使えるか確認（PyTorch）

try:
    import torch
    print("✅ PyTorch インストール成功")
    print(f"PyTorch version: {torch.__version__}")
    
    # CUDA が利用可能か確認
    if torch.cuda.is_available():
        print(f"✅ CUDA が利用可能です")
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU デバイス数: {torch.cuda.device_count()}")
        
        # 現在のGPU情報
        current_device = torch.cuda.current_device()
        print(f"現在のGPU: {torch.cuda.get_device_name(current_device)}")
        
        # GPU メモリ情報
        gpu_memory = torch.cuda.get_device_properties(current_device).total_memory / 1024**3
        print(f"GPU メモリ: {gpu_memory:.2f} GB")
        
        # 簡単なテスト
        test_tensor = torch.tensor([1, 2, 3], device='cuda')
        print(f"GPU テストテンソル: {test_tensor}")
        print("✅ GPU が使用可能です")
    else:
        print("⚠️ CUDA が利用できません。CPU モードで実行します")
        print("GPU が割り当てられていない可能性があります。")
        print("Colab の「ランタイム」→「ランタイムのタイプを変更」でGPUを有効化してください。")
        
except ImportError as e:
    print("❌ PyTorch のインポートに失敗しました")
    print("エラー:", e)
    print("CPU モードで実行します")
except Exception as e:
    print("⚠️ GPU 確認中にエラー:", e)
    import traceback
    traceback.print_exc()

In [ ]:
# 5. 入力フォルダ / 出力フォルダの設定（Google Drive 上）

from pathlib import Path

# TODO: あなたの Drive 上の TIF フォルダに書き換えてください
INPUT_FOLDER = "/content/drive/MyDrive/CT_data"

OUTPUT_BASE = "/content/drive/MyDrive/kenkyuu_results"
Path(OUTPUT_BASE).mkdir(parents=True, exist_ok=True)

print("INPUT_FOLDER =", INPUT_FOLDER)
print("OUTPUT_BASE =", OUTPUT_BASE)


In [ ]:
# 6. （オプション）CPU モードで軽く確認（r=1 だけ）

# まずは r=1 だけで動作確認したい場合はこのセルを実行
# !python scripts/run_full_pipeline_cli.py \
#   --input "$INPUT_FOLDER" \
#   --output "$OUTPUT_BASE/cli_cpu_test_r1" \
#   --max-radius 1 \
#   --connectivity 6 \
#   --threshold-method otsu \
#   --tau-ratio 0.05 \
#   --tau-gain-rel 0.003 \
#   --contacts-min 4 \
#   --contacts-max 10 \
#   --backend cpu

In [ ]:
# 7. GPU モードで r=10 までフル実行（爆速版）

# これがメインの実行セルです
# GPU を使って r=1 から r=10 まで一気に回します

!python scripts/run_full_pipeline_cli.py \
  --input "$INPUT_FOLDER" \
  --output "$OUTPUT_BASE/gpu_full_r10" \
  --max-radius 10 \
  --connectivity 6 \
  --threshold-method otsu \
  --tau-ratio 0.05 \
  --tau-gain-rel 0.003 \
  --contacts-min 4 \
  --contacts-max 10 \
  --backend gpu

In [ ]:
# 8. GPU モードで複数パラメータを一気に試す（PDCA用）

# 二値化パラメータを変えながら、GPU で一気に回して比較します
# 結果の PNG を Drive で見比べて、最適な設定を見つけます

runs = [
    dict(name="gpu_otsu_no_clahe",  threshold="otsu",     clahe=False),
    dict(name="gpu_otsu_clahe",     threshold="otsu",     clahe=True),
    dict(name="gpu_triangle_no_clahe", threshold="triangle", clahe=False),
    dict(name="gpu_triangle_clahe", threshold="triangle", clahe=True),
]

for cfg in runs:
    out_dir = f"{OUTPUT_BASE}/{cfg['name']}"
    print("=" * 60)
    print(f"Running: {cfg['name']}")
    print(f"  threshold={cfg['threshold']}, clahe={cfg['clahe']}")
    print("=" * 60)

    clahe_flag = "--enable-clahe" if cfg["clahe"] else ""

    !python scripts/run_full_pipeline_cli.py \
      --input "$INPUT_FOLDER" \
      --output "$out_dir" \
      --max-radius 10 \
      --connectivity 6 \
      --threshold-method {cfg["threshold"]} \
      --tau-ratio 0.05 \
      --tau-gain-rel 0.003 \
      --contacts-min 4 \
      --contacts-max 10 \
      --backend gpu \
      $clahe_flag
    
    print(f"✅ {cfg['name']} 完了\n")